In [48]:
from flask import Flask
app = Flask(__name__)

# Spring Controller와 동일한 역할 수행
@app.route("/")
def hello():
    return "<h1>Hello Python</h1>"

if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [29/Sep/2021 19:23:30] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Sep/2021 19:23:33] "GET /test HTTP/1.1" 404 -
127.0.0.1 - - [29/Sep/2021 19:23:43] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Sep/2021 19:23:47] "GET /test HTTP/1.1" 404 -


In [ ]:
# 1) https://developers.kakao.com/ 로그인
# 2) 내 애플리케이션 -> 이전에 만들었던 사항 클릭
# 3) 동의항목: 닉네임 - 필수, 카카오계정 - 선택, 카톡 메시지 - 선택
# 4) 카카오 로그인 - 활성화 설정 ON
# 5) Redirect URI에 https://localhost.com 설정
# 6) 플랫폼 - Web 도메인으로 https://www.naver.com과 https://localhost.com 설정
# 7) Kakao API 문서 위치 - https://developers.kakao.com/docs/latest/ko/kakaologin/rest-api
# 8) 주어진 정보 확인
#    GET /oauth/authorize?client_id={REST_API_KEY}&redirect_uri={REDIRECT_URI}&response_type=code HTTP/1.1
#    Host: kauth.kakao.com
#    요약 정보에 있는 REST API 키가 REST_API_KEY 위치에 배치되면 됨
#    REDIRECT_URI의 경우 앞서 등록했던 https://localhost.com
# 9) 해당 URL로 요청을 하면 Kakao API를 활용할 수 있는 토큰을 발급받을 수 있고
#    주어진 시간내에 처리하지 않으면 토큰이 만료되므로 먼저 코드를 작성하도록 한다.

In [46]:
import requests

url = "https://kauth.kakao.com/oauth/token"

#data = {
#    "grant_type" : "authorization_code",
#    "client_id" : "REST_API_키"
#    "redirect_uri" : "https://localhost.com"
#    "code" : "발급받은 토큰값"
#}

data = {
    "grant_type" : "authorization_code",
    "client_id" : "4a0ef335a439753d31b31b38de362400",
    "redirect_uri" : "https://localhost.com",
    "code" : "75841BOvHkwOp2pxi0vkdR5v4FgoXdYhtMWeJa4CKjyabOpdMVov8IdYfOoQfUI21cgdwwo9dGkAAAF8MISbyw"
}

response = requests.post(url, data=data)

if response.status_code != 200:
    print("Error!")
    
else:
    tokens = response.json()
    print(tokens)

Error!


In [49]:
import json
import requests
import datetime
import os

# Kakao Token 저장 파일
KAKAO_TOKEN_SAVED_FILE = "res/kakao_msg/kakao_token.json"

# 저장
def save_tokens(filename, tokens):
    with open(filename, "w") as fp:
        json.dump(tokens, fp)
        
# 읽기
def load_tokens(filename):
    with open(filename) as fp:
        tokens = json.load(fp)
        
    return tokens

# Access Toekn 갱신을 위한 Refresh
def update_tokens(app_key, filename):
    tokens = load_tokens(filename)
    
    url = "https://kauth.kakao.com/oauth/token"
    
    data = {
        "grant_type" : "refresh_token",
        "client_id" : app_key,
        "refresh_token" : tokens['refresh_token']
    }
    
    response = requests.post(url, data=data)
    
    if response.status_code != 200:
        print("Error!")
        tokens = None
    else:
        print(response.json())
        
        # 기존 내용 백업:
        now = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        backup_filename = filename + "." + now
        os.rename(filename, backup_filename)
        
        # 새롭게 업데이트된 토큰 저장
        tokens['access_token'] = response.json()['access_token']
        save_tokens(filename, tokens)
        
    return tokens

# token 저장
save_tokens(KAKAO_TOKEN_SAVED_FILE, tokens)

In [44]:
# Kakao Talk Message 보내기

import json
import requests

# 토큰 정보 활용
tokens = load_tokens(KAKAO_TOKEN_SAVED_FILE)

# 메세지 보내기 URL
url = "https://kapi.kakao.com/v2/api/talk/memo/default/send"

# requests parameter 설정
headers = {
    # Authorization: Bearer {ACCESS_TOKEN}
    "Authorization" : "Bearer " + tokens['access_token']
}

data = {
    "template_object" : json.dumps({
        "object_type" : "text",
        "text" : "지금 집에 도둑이 들어왔어요. 당신의 물품이 다 털렸습니다. 가까운 경찰서 고고",
        "link" : {
            # 이동할 링크 정보니까 사이트 주소를 걸면 더 좋음
            "web_url" : "www.naver.com"
        }
    })
}

# 메세지 전송
response = requests.post(url, headers=headers, data=data)
print(response.status_code)

if response.status_code != 200:
    print("Error!")
else:
    print("카톡 메세지가 간드아!!!")

200
카톡 메세지가 간드아!!!


In [45]:
from flask import Flask, url_for
app = Flask(__name__)

# Spring Controller와 동일한 역할 수행
@app.route("/")
def hello():
    return "<h1>Hello Python</h1>"

@app.route("/member/<id>")
def get_member(id):
    return 'member : ' + id

if __name__ == "__main__":
    with app.test_request_context():
        print(url_for('hello'))
        print(url_for('get_member', id='eddi'))

/
/member/eddi


In [ ]:
from flask import Flask, url_for, request
app = Flask(__name__)

# Spring Controller와 동일한 역할 수행
@app.route("/")
def hello():
    return "<h1>Hello Python</h1>"

@app.route("/member/<id>")
def get_member(id):
    return 'member : ' + id

@app.route("/test", methods=['POST', 'GET'])
def chk_get_post(entity=None):
    err = None
    
    if request.method == 'POST':
        id = request.form['id']
        email = request.form['email']
        
        if not id and not email:
            return add_entity(request.form)
    else:
        err = "Invalid!!!"
        
    return err

if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [29/Sep/2021 19:24:15] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Sep/2021 19:24:19] "GET /test HTTP/1.1" 200 -


In [ ]:
# Cors 문제 해결
# pip install flask_cors
from flask import Flask, url_for, request, jsonify
from flask_cors import CORS, cross_origin

app = Flask(__name__)
CORS(app)

# Spring Controller와 동일한 역할 수행
@app.route("/")
def hello():
    return "<h1>Hello Python</h1>"

@app.route("/member/<id>")
def get_member(id):
    return 'member : ' + id

@app.route("/login", methods=['POST', 'GET'])
def chk_get_post(entity=None):
    err = None
    
    if request.method == 'POST':
        print("POST 처리")
        #userId = request.form['userId']
        #pw = request.form['password']
        #print("userId = {0}, pw = {1}".format(userId, pw))
        # request.get_json() 상대편에서 보낸 json 내용을 읽음
        data = request.get_json()
        print(data)
        
        # 여기에 카톡 Open API 연동하거나 딥러닝을 활용하는 것임
        # 물론 우리는 시간이 부족해서 딥러닝이 불가함
        # 그러므로 카톡 Open API나 주식 API 등등 여러가지 Open API들을 활용하면 될 것임
        
        #return "userId = " + userId + ", password = " + pw
        # json 데이터를 전송
        return jsonify(data)
    else:
        print("GET 처리")
        err = "Invalid!!!"
        
    return err

if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [29/Sep/2021 19:52:15] "OPTIONS /login HTTP/1.1" 200 -
127.0.0.1 - - [29/Sep/2021 19:52:15] "POST /login HTTP/1.1" 200 -


POST 처리
{'userId': 'test', 'password': 'test'}


127.0.0.1 - - [29/Sep/2021 19:55:15] "OPTIONS /login HTTP/1.1" 200 -
127.0.0.1 - - [29/Sep/2021 19:55:15] "POST /login HTTP/1.1" 200 -


POST 처리
{'userId': 'test', 'password': 'test'}
